In [ ]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = 'c9970cdd40e5470baa2eefb0deb5ed82'
CLIENT_SECRET = '4170e764c9df47abbf782fa250c953bb'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

#connection timeout correction
import urllib3, socket
from urllib3.connection import HTTPConnection

HTTPConnection.default_socket_options = (HTTPConnection.default_socket_options + [
    (socket.SOL_SOCKET, socket.SO_SNDBUF, 1000000), #1MB in byte
            (socket.SOL_SOCKET, socket.SO_RCVBUF, 1000000)
        ])

Access token obtained successfully.


In [ ]:
pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 6.7 MB/s eta 0:00:00


In [1]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 33.1 MB/s eta 0:00:00


In [7]:
import pandas as pd

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth


def get_trending_playlist_data(username,playlist_id):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    # Iterate through all pages of results
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in tracks: # Iterate over the combined tracks
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track_Name': track_name,
            'Artists': artists,
            'Album_Name': album_name,
            'Album_ID': album_id,
            'Track_ID': track_id,
            'Popularity': popularity,
            'Release_Date': release_date,
            'Duration(ms)': audio_features['duration_ms'] if audio_features else None,
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key_note': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [ ]:
from IPython.display import display
playlist_id = '5S1DXXeJr9mKHHjRqKKjcR'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data('Spotify',playlist_id)

In [ ]:
# Display the DataFrame
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track_Name        110 non-null    object 
 1   Artists           110 non-null    object 
 2   Album_Name        110 non-null    object 
 3   Album_ID          110 non-null    object 
 4   Track_ID          110 non-null    object 
 5   Popularity        110 non-null    int64  
 6   Release_Date      110 non-null    object 
 7   Duration(ms)      110 non-null    int64  
 8   Danceability      110 non-null    float64
 9   Energy            110 non-null    float64
 10  Key_note          110 non-null    int64  
 11  Loudness          110 non-null    float64
 12  Mode              110 non-null    int64  
 13  Speechiness       110 non-null    float64
 14  Acousticness      110 non-null    float64
 15  Instrumentalness  110 non-null    float64
 16  Liveness          110 non-null    float64
 1

In [ ]:
music_df.duplicated().sum()

0

In [ ]:
df=music_df.drop_duplicates(subset=['Track_ID'], keep='first')


In [23]:
import mysql.connector

mydb = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "     " #Use your username",
  password = "     " #Use your password,
  database = "    " #Add your database name
)

In [24]:
cursor=mydb.cursor()

In [ ]:

create='''create table if not exists Tamil_songs(Track_Name VARCHAR(100), Artists VARCHAR(300), Album_Name VARCHAR(200),
          Album_ID VARCHAR(100), Track_ID VARCHAR(100),
          Popularity INT, Release_Date DATETIME, Duration_ms BIGINT,
          Danceability FLOAT, Energy FLOAT, Key_note INT,
          Loudness FLOAT, Mode INT, Speechiness FLOAT,
          Acousticness FLOAT, Instrumentalness FLOAT,
          Liveness FLOAT, Valence FLOAT, Tempo FLOAT,
          PRIMARY KEY (Track_ID)  )'''

cursor.execute(create)
mydb.commit()

In [ ]:
for i,j in df.iterrows():
    insert='''insert ignore into Tamil_songs(Track_Name, Artists, Album_Name,
          Album_ID, Track_ID,
          Popularity, Release_Date, Duration_ms,
          Danceability, Energy, Key_note,
          Loudness, Mode, Speechiness,
          Acousticness, Instrumentalness,
          Liveness, Valence, Tempo)values(%s,%s,%s,%s,%s,%s,%s,
                                          %s,%s,%s,%s,%s,%s,%s,
                                          %s,%s,%s,%s,%s)'''

    values=(j['Track_Name'], j['Artists'], j['Album_Name'],
          j['Album_ID'], j['Track_ID'], j['Popularity'],
          j['Release_Date'], j['Duration(ms)'], j['Danceability'],
          j['Energy'], j['Key_note'], j['Loudness'], j['Mode'],
          j['Speechiness'], j['Acousticness'], j['Instrumentalness'],
          j['Liveness'], j['Valence'], j['Tempo'])
    cursor.execute(insert,values)
    mydb.commit()


In [25]:
cursor.execute("SELECT * FROM Tamil_songs")
result = cursor.fetchall()
songs_df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

In [14]:
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3602 entries, 0 to 3601
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Track_Name        3602 non-null   object        
 1   Artists           3602 non-null   object        
 2   Album_Name        3602 non-null   object        
 3   Album_ID          3602 non-null   object        
 4   Track_ID          3602 non-null   object        
 5   Popularity        3602 non-null   int64         
 6   Release_Date      3101 non-null   datetime64[ns]
 7   Duration_ms       3602 non-null   int64         
 8   Danceability      3602 non-null   float64       
 9   Energy            3602 non-null   float64       
 10  Key_note          3602 non-null   int64         
 11  Loudness          3602 non-null   float64       
 12  Mode              3602 non-null   int64         
 13  Speechiness       3602 non-null   float64       
 14  Acousticness      3602 n

In [30]:
songs_df.to_csv('all_songs.csv', index=False)

In [ ]:
songs_df.isnull().sum()

,0
Track_Name,0
Artists,0
Album_Name,0
Album_ID,0
Track_ID,0
Popularity,0
Release_Date,0
Duration_ms,0
Danceability,0
Energy,0


In [ ]:
print(songs_df.isnull().sum())

Track_Name          0
Artists             0
Album_Name          0
Album_ID            0
Track_ID            0
Popularity          0
Release_Date        0
Duration_ms         0
Danceability        0
Energy              0
Key_note            0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = songs_df

In [ ]:
def calculate_weighted_popularity(release_date):

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [ ]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = songs_df[['Danceability', 'Energy', 'Key_note',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=10):
    if input_song_name not in songs_df['Track_Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = songs_df[songs_df['Track_Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = songs_df.iloc[similar_song_indices][['Track_Name', 'Artists', 'Album_Name', 'Release_Date', 'Popularity']]

    return content_based_recommendations

In [ ]:
def hybrid_recommendations(input_song_name, num_recommendations=10, alpha=0.5):
    if input_song_name not in songs_df['Track_Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = songs_df.loc[songs_df['Track_Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        songs_df.loc[songs_df['Track_Name'] == input_song_name, 'Release_Date'].values[0]
    )

    new_entry = pd.DataFrame({
        'Track_Name': [input_song_name],
        'Artists': [songs_df.loc[songs_df['Track_Name'] == input_song_name, 'Artists'].values[0]],
        'Album_Name': [songs_df.loc[songs_df['Track_Name'] == input_song_name, 'Album_Name'].values[0]],
        'Release_Date': [songs_df.loc[songs_df['Track_Name'] == input_song_name, 'Release_Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track_Name'] != input_song_name]

    return hybrid_recommendations

In [ ]:
input_song_name = "Rathathin Rathamay"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=10)
print(f"Hybrid recommended songs for '{input_song_name}':")
display(recommendations)

Hybrid recommended songs for 'Rathathin Rathamay':


,Track_Name,Artists,Album_Name,Release_Date,Popularity
1,Uyirin Uyire,"KK, K. S. Chithra",Kaakha Kaakha (Original Motion Picture Soundtr...,2003-09-27 00:00:00,57.0
2,Open the Tasmac,"Anirudh Ravichander, Deva",Maan Karate Special (Original Motion Picture S...,2014-07-11 00:00:00,55.0
6,Arugil,"Harris Jayaraj, Hariharan, Tanvi Shah",Dhruva Natchathiram (Original Motion Picture S...,2023-11-24 00:00:00,23.0
0,Baaga Unnara,"D. Imman, Udit Narayan, SUVI","Naan Avan Illai, 2 (Original Motion Picture So...",2009-10-30 00:00:00,19.0
3,"Open the Tasmac (From ""Maan Karate"")","Anirudh Ravichander, Deva",Nanba Nanba! Songs That Celebrate Friendship,2014-07-30 00:00:00,17.0
9,Ithu thana (From “Saamy”),K. S. Chithra,"Nilave Vaa, Vol. 1",0001-01-01 00:00:00,11.0
4,"Papara Mittai (From ""R. K. Nagar"")",Gaana Guna,"Papara Mittai (From ""R. K. Nagar"")",2018-02-14 00:00:00,0.0
5,Uyirile,"Srinivas, Mahalakshmi Iyer",Vettaiaadu Vilayadu (Original Motion Picture S...,2006-03-08 00:00:00,0.0
7,Idhuthaanaa,"Harris Jayaraj, K. S. Chithra",Saamy (Original Motion Picture Soundtrack),2003-05-01 00:00:00,0.0
8,Puliya Kili Jeyicha,Harish Raghavendra,Veerappu (Original Motion Picture Soundtrack),2009-11-22 00:00:00,0.0
